In [2]:
import requests
import os
import time
import urllib
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "root"))
session = driver.session()

# Add uniqueness constraints.
session.run( "CREATE CONSTRAINT ON (p:PartidoPolitico) ASSERT p.name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (pr:Presidente) ASSERT pr.name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (v:Vicepresidente) ASSERT v.name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (a:Asambleistas) ASSERT a.name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (pa:PartidosPoliticosAliados) ASSERT pa.name IS UNIQUE;")

session.run( "CREATE CONSTRAINT ON (t:Tweet) ASSERT t.id IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (u:User) ASSERT u.screen_name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (h:Hashtag) ASSERT h.name IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (l:Link) ASSERT l.url IS UNIQUE;")
session.run( "CREATE CONSTRAINT ON (s:Source) ASSERT s.name IS UNIQUE;")



# Consigue Twitter bearer Twitter para pasar al encabezado.
TWITTER_BEARER = "AAAAAAAAAAAAAAAAAAAAAOUfKwEAAAAAhD1GxFC4qrdb23QGSbz3io6s3jI%3D9JDdyPjXIMgQfYi0IEX7ULjGO5WKvkXiTHWfXApMe7Y8dX4y1d"

# URL parameters.
q = urllib.parse.quote_plus(os.environ.get("TWITTER_SEARCH","@LassoGuillermo OR #AsambleístasDelCAMBIO OR #CREO21 OR #CapacidadParaCambiar OR #LassoBorrero2021 OR #LassoPresidente OR #YoCREOenelCambio OR @BloqueCREO21 OR #Todo21 OR #ElAñoDelCAMBIO OR @La6Ecuador OR @ABorreroVega OR @BernardaOM OR @PepeJara21 OR @CREOAzuay"))

count = 100
result_type = "recent"
lang = "es"
since_id = -1

while True:
    try:
        print(q)
        # Build URL.
        url = "https://api.twitter.com/1.1/search/tweets.json?q=%s&count=%s&result_type=%s&lang=%s&since_id=%s" % (q, count, result_type, lang, since_id)
        # Send GET request.
        r = requests.get(url, headers = {"accept":"application/json","Authorization":"Bearer " + TWITTER_BEARER})

        # Keep status objects.
        tweets = r.json()["statuses"]
        partidopolitico = r.json()["statuses"]

        if tweets:
            plural = "s." if len(tweets) > 1 else "."
            print("Found " + str(len(tweets)) + " tweet" + plural)
        else:
            print("No tweets found.\n")
            time.sleep(65)
            continue

        # Metodo que pasa a cyber e imprime la consulta
        query = """
        UNWIND $tweets AS t
        WITH t
        ORDER BY t.id
        WITH t,
             t.entities AS e,
             t.user AS u,
             t.retweeted_status AS retweet
        MERGE (tweet:Tweet {id:t.id})
        SET tweet.text = t.text,
            tweet.created_at = t.created_at,
            tweet.favorites = t.favorite_count
        MERGE (user:User {screen_name:u.screen_name})
        SET user.name = u.name,
            user.location = u.location,
            user.followers = u.followers_count,
            user.following = u.friends_count,
            user.statuses = u.statusus_count,
            user.profile_image_url = u.profile_image_url
        MERGE (user)-[:POSTS]->(tweet)
        MERGE (source:Source {name:t.source})
        MERGE (tweet)-[:USING]->(source)
        FOREACH (h IN e.hashtags |
          MERGE (tag)-[:TAGS]->(tweet)
        )
        FOREACH (u IN e.urls |
          MERGE (url:Link {url:u.expanded_url})
          MERGE (tweet)-[:CONTAINS]->(url)
          MERGE (tweet)-[:pertenece]->(partidopolitico)
        )
        FOREACH (m IN e.user_mentions |
          MERGE (mentioned:User {screen_name:m.screen_name})
          ON CREATE SET mentioned.name = m.name
          MERGE (tweet)-[:MENTIONS]->(mentioned)
        )
        FOREACH (r IN [r IN [t.in_reply_to_status_id] WHERE r IS NOT NULL] |
          MERGE (reply_tweet:Tweet {id:r})
          MERGE (tweet)-[:REPLY_TO]->(reply_tweet)
        )
        FOREACH (retweet_id IN [x IN [retweet.id] WHERE x IS NOT NULL] |
            MERGE (retweet_tweet:Tweet {id:retweet_id})
            MERGE (tweet)-[:RETWEETS]->(retweet_tweet)
        )
        """

        # Send Cypher query.
        session.run(query,{'tweets':tweets})
        print("Tweets added to graph!\n")
        time.sleep(65)

    except Exception as e:
        print(e)
        time.sleep(65)
        continue

%40LassoGuillermo+OR+%23Asamble%C3%ADstasDelCAMBIO+OR+%23CREO21+OR+%23CapacidadParaCambiar+OR+%23LassoBorrero2021+OR+%23LassoPresidente+OR+%23YoCREOenelCambio+OR+%40BloqueCREO21+OR+%23Todo21+OR+%23ElA%C3%B1oDelCAMBIO+OR+%40La6Ecuador+OR+%40ABorreroVega+OR+%40BernardaOM+OR+%40PepeJara21+OR+%40CREOAzuay
Found 100 tweets.
Tweets added to graph!

%40LassoGuillermo+OR+%23Asamble%C3%ADstasDelCAMBIO+OR+%23CREO21+OR+%23CapacidadParaCambiar+OR+%23LassoBorrero2021+OR+%23LassoPresidente+OR+%23YoCREOenelCambio+OR+%40BloqueCREO21+OR+%23Todo21+OR+%23ElA%C3%B1oDelCAMBIO+OR+%40La6Ecuador+OR+%40ABorreroVega+OR+%40BernardaOM+OR+%40PepeJara21+OR+%40CREOAzuay
Found 100 tweets.
Tweets added to graph!

%40LassoGuillermo+OR+%23Asamble%C3%ADstasDelCAMBIO+OR+%23CREO21+OR+%23CapacidadParaCambiar+OR+%23LassoBorrero2021+OR+%23LassoPresidente+OR+%23YoCREOenelCambio+OR+%40BloqueCREO21+OR+%23Todo21+OR+%23ElA%C3%B1oDelCAMBIO+OR+%40La6Ecuador+OR+%40ABorreroVega+OR+%40BernardaOM+OR+%40PepeJara21+OR+%40CREOAzuay


[WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))


Found 100 tweets.
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))
%40LassoGuillermo+OR+%23Asamble%C3%ADstasDelCAMBIO+OR+%23CREO21+OR+%23CapacidadParaCambiar+OR+%23LassoBorrero2021+OR+%23LassoPresidente+OR+%23YoCREOenelCambio+OR+%40BloqueCREO21+OR+%23Todo21+OR+%23ElA%C3%B1oDelCAMBIO+OR+%40La6Ecuador+OR+%40ABorreroVega+OR+%40BernardaOM+OR+%40PepeJara21+OR+%40CREOAzuay
Found 100 tweets.
Failed to read from closed connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))
%40LassoGuillermo+OR+%23Asamble%C3%ADstasDelCAMBIO+OR+%23CREO21+OR+%23CapacidadParaCambiar+OR+%23LassoBorrero2021+OR+%23LassoPresidente+OR+%23YoCREOenelCambio+OR+%40BloqueCREO21+OR+%23Todo21+OR+%23ElA%C3%B1oDelCAMBIO+OR+%40La6Ecuador+OR+%40ABorreroVega+OR+%40BernardaOM+OR+%40PepeJara21+OR+%40CREOAzuay
Found 100 tweets.
Failed to read from closed connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))
%40LassoGuillermo+OR+

KeyboardInterrupt: 

In [ ]:
#Node2Vec
MATCH (t1:Tweet {name: 'Guillermo Lasso'})-[tweet1:MENTIONS]->(mentioned)
MATCH (t2:Tweet {name: "Andrea Rodriguez Carranza"})-[tweet2:MENTIONS]->(mentioned)
RETURN t1.name AS from,
       t2.name AS to,
       algo.similarity.euclideanDistance(collect(tweet1.score), collect(tweet2.score)) AS similarity
#Euclidean Similarity
MATCH (t1:Person {name: 'Guillermo Lasso'})-[tweet1:MENTIONS]->(mentioned)
MATCH (t2:Person {name: "Andrea Rodriguez Carranza"})-[tweet2:MENTIONS]->(mentioned)
RETURN t1.name AS from,
       t2.name AS to,
       algo.similarity.euclideanDistance(collect(tweet1.score), collect(tweet2.score)) AS similarity
        
#Resource Allocation
MATCH (t1:Tweet {name: 'Guillermo Lasso'})
MATCH (t2:Tweet {name: 'Andrea Rodriguez Carranza'})
RETURN algo.linkprediction.resourceAllocation(t1, t2) AS score
#Depth First Search

CALL gds.alpha.dfs.stream(
  graphName: String,
  configuration: Map
)
YIELD
  // general stream return columns
  startNodeId: Integer,
          nodeIds: Integer,
                  path: Path
CALL gds.graph.create('myGraph', 'Node', 'REL', { relationshipProperties: 'cost' })